<a href="https://colab.research.google.com/github/Himanshu-Tagde/Assignment/blob/main/Aadhar_Cards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract openpyxl


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:

uploaded = files.upload()

Saving a1.jpg to a1 (5).jpg
Saving a2.jpg to a2 (5).jpg
Saving a3.jpeg to a3 (5).jpeg
Saving a4.jpg to a4 (5).jpg
Saving a5.jpg to a5 (5).jpg
Saving a6.jpg to a6 (5).jpg
Saving a7.jpg to a7 (5).jpg
Saving a8.jpg to a8 (5).jpg
Saving a9.jpg to a9 (5).jpg
Saving a10.png to a10 (5).png


In [ ]:
import cv2
import pytesseract
from PIL import Image
import numpy as np
import re
import pandas as pd
import os

pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, None, fx=2.5, fy=2.5, interpolation=cv2.INTER_LINEAR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    denoised = cv2.fastNlMeansDenoising(gray, h=30)
    _, thresh = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh

# Extract information from image text
def extract_details(text):
    lines = [line.strip() for line in text.split("\n") if line.strip()]
    full_text = " ".join(lines)

    name = "Not Found"
    dob = "Not Found"
    gender = "Not Found"
    aadhaar = "Not Found"
    pin_code = "Not Found"
    state = "Not Found"
    address = "Not Found"

    # Aadhaar Number
    aadhaar_match = re.findall(r'\b\d{4}\s\d{4}\s\d{4}\b', full_text)
    if aadhaar_match:
        aadhaar = aadhaar_match[0]

    # DOB
    dob_match = re.search(r'(DOB[:\s]*)?(\d{2}/\d{2}/\d{4})', full_text)
    if dob_match:
        dob = dob_match.group(2)

    # Gender
    gender_match = re.search(r'\b(Male|Female|Other)\b', full_text, re.IGNORECASE)
    if gender_match:
        gender = gender_match.group(0).capitalize()

    # PIN Code
    pin_match = re.search(r'\b\d{6}\b', full_text)
    if pin_match:
        pin_code = pin_match.group(0)

    # Name
    for line in lines:
        if re.match(r'^[A-Z][a-zA-Z\s]+$', line) and len(line.split()) >= 2:
            name = line
            break

    # Address
    possible_address = [line for line in lines if len(line) > 20 and not re.search(r'\d{4}\s\d{4}', line)]
    if possible_address:
        address = possible_address[0]
        state_match = re.search(r'\b[A-Za-z]+\b$', address)
        if state_match:
            state = state_match.group(0)

    return {
        "Name": name,
        "DOB": dob,
        "Gender": gender,
        "Aadhaar Number": aadhaar,
        "Address": address,
        "State": state,
        "PIN Code": pin_code
    }

# Main function for processing one image
def process_aadhaar_image(image_path):
    print(f"Processing: {image_path}")
    img = preprocess_image(image_path)
    text = pytesseract.image_to_string(img, config='--oem 3 --psm 6')
    return extract_details(text)

# Get all image filenames uploaded
image_paths = list(uploaded.keys())

# Process each image
all_data = []
for img_file in image_paths:
    result = process_aadhaar_image(img_file)
    result['Filename'] = img_file
    all_data.append(result)

# Convert to DataFrame
df = pd.DataFrame(all_data)

# Save as Excel
excel_path = "Extracted_Aadhaar_Data.xlsx"
df.to_excel(excel_path, index=False)

# Download the file
files.download(excel_path)


Processing: a1 (5).jpg
Processing: a2 (5).jpg
Processing: a3 (5).jpeg
Processing: a4 (5).jpg
Processing: a5 (5).jpg
Processing: a6 (5).jpg
Processing: a7 (5).jpg
Processing: a8 (5).jpg
Processing: a9 (5).jpg
Processing: a10 (5).png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>